In [1]:
import io
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer
import re

def yield_tokens(file_path, tokenizer):
    with io.open(file_path, encoding = 'utf-8') as file:
        for line in file:
            if re.search('[a-zA-Z]', line):
                yield tokenizer(line.lower().replace('\n', '').strip())
                

/Users/garrettpartenza/.pyenv/versions/transformer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
textfile = "data/discourses.mb.txt"
tokenizer = get_tokenizer("spacy")
contains_chars = lambda x: re.search('[a-zA-Z]', x)

vocab = build_vocab_from_iterator(yield_tokens(textfile, tokenizer), specials=["<unk>", "<eos>", "<sos>", "<pad>"])
vocab.set_default_index(-1)

/Users/garrettpartenza/.pyenv/versions/transformer/lib/python3.9/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [3]:
input_sequences = []

with io.open(textfile, encoding = 'utf-8') as file:
    for line in file.readlines():
        if contains_chars(line):
            line = line.lower().replace('\n', '').strip()
            words = tokenizer(line)
            tokens = [vocab[word] for word in words]
            for i in range(1, len(tokens)): 
                n_gram_seqs = tokens[:i+1]
                input_sequences.append(n_gram_seqs)

In [4]:
num_tokens = len(vocab)
d_model = 16
dropout = 0.1
seq_len = 64

In [5]:
import torch.nn.functional as F

def padding(arr, seq_len, mode="right"):
    assert mode=="right" or mode=="left", "invalid padding mode"
    if mode=="right":
        return F.pad(arr, (0, seq_len-arr.nelement()))
    else:
        return F.pad(arr, (seq_len-arr.nelement()), 0)

In [6]:
import torch
import torch.nn as nn

luc = nn.Embedding(num_tokens, d_model)

In [7]:
example = input_sequences[-1]
example_padded = padding(torch.tensor(example), seq_len)
embedding_func = lambda x: luc(torch.tensor(x, dtype=torch.long))
example_embedding = torch.tensor([luc(token_num).tolist() for token_num in example_padded])
example_embedding.shape

torch.Size([64, 16])

In [8]:
import numpy as np
import matplotlib.pyplot as plt
 
def positional_encoding(seq_len, d, n=10000):
    p = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            p[k, 2*i] = np.sin(k/denominator)
            p[k, 2*i+1] = np.cos(k/denominator)
    return p

p_embedding = torch.tensor(positional_encoding(seq_len=seq_len, d=d_model, n=10000))
p_embedding.shape

torch.Size([64, 16])

In [9]:
rep = torch.add(example_embedding, p_embedding)
rep.shape

torch.Size([64, 16])

In [10]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        # all h heads are calculated in a single matrix multiplication (better parallelization)
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            # reshapes into the individual attention heads of shape h x d_model // h
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]
        

        # 2) Apply attention on all the projected vectors in batch.
        # calculate attention using paper formula (scaled dot product attention to prevent small gradients)
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )
        
        # 3) "Concat" using a view and apply a final linear.
        # concats all attention layers back into d_model size feature vector
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        
            
        del query
        del key
        del value
        
        # applys fourth linear transformation
        return self.linears[-1](x)

In [11]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        # attention over itself
        self.self_attn = self_attn
        # attention over encoder
        self.feed_forward = feed_forward
        # three sublayers now due to encoder-decoder attention
        self.sublayer = clones(SublayerConnection(size, dropout), 2)

    def forward(self, x):
        "Follow Figure 1 (right) for connections."
        # same as the encoder layer, plus an additional sublayer for the encoder-decoder attention
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x))
        return self.sublayer[1](x, self.feed_forward)

In [12]:
# wrapper to apply residual connection addition as well as layer normalization
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [13]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        # bunch of math, tldr; rescales features with mean of 0 and std of 1 (paper -> https://arxiv.org/abs/1607.06450)
        # not sure what the function of eps is here
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [14]:
# literally the same high level wrapper as encoder, where differences occur within the actual decoder layer class
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)

In [15]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [16]:
# simple single hidden layer feed forward nueral network with relu acitvation funcftion
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [17]:
import copy
import math

h = 2
N = 1
d_ff = 64
c = copy.deepcopy
attn = MultiHeadedAttention(h, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
gpt = Decoder(DecoderLayer(d_model, c(attn), c(ff), dropout), N)

In [18]:
for p in gpt.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [19]:
print("Input shape: ", rep.shape)
out = gpt(rep.float())
out.shape

Input shape:  torch.Size([64, 16])


torch.Size([64, 64, 16])

In [20]:
from torch.nn.functional import log_softmax

class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        # matmul(1xd_model, d_modelxvocav) gives a 1 dimensional array of softmaxed values for each vocab word
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [21]:
generator = Generator(d_model, num_tokens)

In [22]:
prob = generator(out[:, -1])

In [23]:
prob.shape

torch.Size([64, 5767])

In [24]:
_, next_word = torch.max(prob, dim=1)

In [25]:
next_word.shape

torch.Size([64])

In [26]:
next_word.shape

torch.Size([64])

In [27]:
next_word = next_word.data[0]

In [28]:
next_word

tensor(1705)

In [29]:
vocab.lookup_token(next_word)

'tame'